In [10]:
import os
import cv2
import shutil
import requests
from configparser import ConfigParser

# Config

In [11]:
parser = ConfigParser()
parser.read('dev.ini')
#print(parser.sections())
#parser.options('settings')
end_folder_name = parser.get('settings', 'dest_name')
create_folder = parser.getboolean('settings', 'create_folder', fallback = True)
path= parser.get('settings', 'path', fallback = os.getcwd() + '\\' + end_folder_name)
th = parser.getint('settings', 'size_treshold', fallback = 9999)
timeout = parser.getint('settings', 'timeout', fallback = 20)
clean_src = parser.getboolean('settings', 'clean_src', fallback = False)
neuron_number = parser.getint('settings', 'neuron_number', fallback = 0)

In [12]:
path

'C:\\Users\\Julian\\Documents\\Studium\\Delft\\Deep Learning\\data'

'C:\\Users\\Julian\\Documents\\Studium\\Delft\\Deep Learning\\data'

# Get images and cut+resize

In [13]:
if create_folder:
    try:
        dest_folder_path = os.mkdir(os.path.join(path,end_folder_name))
    except:
        dest_folder_path = os.path.join(path,end_folder_name)

In [14]:
#Cutting image function
def cutting(img):
    
    width, height = img.shape[:2]
    
    if width >= th or height>= th:
        pass
    else:
        if width > height:
            x = int((width - height)/2)
            img = img[x:x+height, 0:height]
        else:
            x = int((height - width)/2)
            img = img[0:width, x:x + width] 
    
    return img   

In [15]:
content = []
with open(os.path.join(path,'url.txt'),encoding="utf8") as ff:
    content = ff.readlines()

In [16]:
#Delete duplicates
content = list(set(content))

In [19]:
i = 0

for num, url in enumerate(content): 
        
    try:
        img_data = requests.get(url, timeout = timeout).content
        
    except requests.exceptions.RequestException as e:
        print(e)
        
    try:
        
        i = str(num).zfill(5)
        name = os.path.join(os.path.join(path,'Source'), str(i) + '.JPEG')
    
    
        with open(name, 'wb') as handler:
            handler.write(img_data)
  

        print(name)

        img = cv2.imread(name)
        
    
        img = cutting(img)

        
        res = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)

        cv2.imwrite(os.path.join(dest_folder_path, str(i) + '.JPEG'),res)
        
 
    except Exception as e:
        print(e)

C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00000.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00000.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00001.JPEG
'NoneType' object has no attribute 'shape'
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00001.JPEG
'NoneType' object has no attribute 'shape'
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00002.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00002.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00003.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00003.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00004.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00004.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\00005.JPEG
C:\Users\Julian\Documents\Studium\Delft\Deep Learning\data\Source\0

In [ ]:
ff.close()

# Create Name File

*First do manual cleaning*

In [ ]:
user_input = input('Did you clean pics manualy? [y/n]:')

In [ ]:
if user_input == 'y':
    content = os.listdir(dest_folder_path)
    #Create txt file to store image names
    f= open(dest_folder_path + '\\' + end_folder_name + "_img_names.txt","w+")
    for num, address in enumerate(content):
        i = str(num+1).zfill(5)
        f.write(end_folder_name + '_' +  i + '.JPEG ' + str(neuron_number) + ' \n')

In [ ]:
f.close()

In [9]:
if clean_src:
    p = os.path.join(path,'Source')
    for item in os.listdir(p):
        os.remove(os.path.join(p, item))